# Evaluation Concepts

## Setup

In those notebook, we'll be evaluating this multiagent

![Multiagent](../images/multiagent.png)

In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

/Users/robertxu/Desktop/Projects/education/eval-concepts


To start, let's load our environment variables from our .env file. Make sure all of the keys necessary in .env.example are included!


In [2]:
from dotenv import load_dotenv
load_dotenv(dotenv_path=".env", override=True)

True

In [3]:
from agents.multi_basic import multiagent, supervisor # Import multiagent helper

from langsmith import Client # Set up LangSmith client
client = Client()

from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o", temperature=0)

## Implementing Evaluations


To recap, Evaluations are made up of three components:

1. A **dataset test** inputs and expected outputs.
2. An **application or target function** that defines what you are evaluating, taking in inputs and returning the application output
3. **Evaluators** that score your target function's outputs.

![Evaluation](../images/evals.png) 

There are many ways you can evaluate an agent. In our ```email_basic.ipynb``` notebook, we cover the basic types. For this multiagent notebook, we'll be showing how to create multiturn evaluations.

### Evaluating Across Multiple Turns

Many LLM applications run across multiple conversation turns with a user. While running end-to-end, single step, and trajectory evaluations can evaluate one given turn in a thread, obtaining a representative example thread of messages can be difficult.

To help judge your application's performance over multiple interactions, OpenEvals includes a `run_multiturn_simulation` method (and its Python async counterpart `run_multiturn_simulation_async`) for simulating interactions between our app and an end user to help evaluate our app's performance from start to finish.

![trajectory](../images/multi_turn.png) 

#### 1. Create a Dataset

To simulate multi-turn conversations, we will create `persona` as the input value to our dataset, which includes information & prompt of the profile of our simulated uers.  
For reference outputs, we will create a `success_criteria`, which will allow our LLM as a judge determine if the conversation was resolved based on the specific criteria. 

In [4]:
# Create a dataset
examples = [
    {
        "persona": "You are a user who is frustrated with your most recent purchase, and wants to get a refund but couldn't find the invoice ID or the amount, and you are looking for the ID. Your customer id is 30. Only provide information on your ID after being prompted.",
        "success_criteria": "Find the invoice ID, which is 333. Total Amount is $8.91."
    },
    {
        "persona": "Your phone number is +1 (204) 452-6452. You want to know the information of the employee who helped you with the most recent purchase.",
        "success_criteria": "Find the employee with the most recent purchase, who is Margaret, a Sales Support Agent with email at margaret@chinookcorp.com. "
    },
    {
        "persona": "Your account ID is 3. You want to learn about albums that the store has by Amy Winehouse.",
        "success_criteria": "The agent should provide the two albums in store, which are Back to Black and Frank by Amy Winehouse."
    },
    {
        "persona": "Your account ID is 10. You want to learn about how to become the best tennis player in the world.",
        "success_criteria": "The agent should avoid answering the question."
    },
]

dataset_name = "LangGraph 101 Multi-Agent: Multi-Turn"

if not client.has_dataset(dataset_name=dataset_name):
    dataset = client.create_dataset(dataset_name=dataset_name)
    client.create_examples(
        inputs=[{"persona": ex["persona"]} for ex in examples],
        outputs=[{"success_criteria": ex["success_criteria"]} for ex in examples],
        dataset_id=dataset.id
    )

#### 2. Define the Application Logic to Evaluate 

To run a multi-turn simulation, we will be leveraging the `run_multiturn_simulation`util in openevals. 

There are a few components to `run_multiturn_simulation`:
- `app`: Our application, or a function wrapping it. Must accept a chat message (dict with "role" and "content" keys) as an input arg and a thread_id as a kwarg. Returns a chat message as output with at least role and content keys.
- `user`: The simulated user. Must accept the current trajectory as a list of messages as an input arg and kwargs for thread_id and turn_counter. Should accept other kwargs as more may be added in future releases. Returns a chat message as output. May also be a list of string or message responses.
- `max_turns`/`maxTurns`: The maximum number of conversation turns to simulate.
- `stopping_condition`/`stoppingCondition`: Optional callable that determines if the simulation should end early. Takes the current trajectory as a list of messages as an input arg and a kwarg named turn_counter, and should return a boolean. We will showing an example of this implementation today!

First, we need to create the `app`, which is our **graph logic** - invoking the graph, and obtaining the most recent message. 

In [5]:
from openevals.llm import create_async_llm_as_judge
from openevals.simulators import run_multiturn_simulation_async, create_llm_simulated_user

graph = multiagent

# Runs the graph and outputs most recent message  
async def run_graph(inputs, thread_id: str):
    """Run graph and track the final response."""
    configuration = {"thread_id": thread_id}

    # Invoke graph until interrupt 
    result = await graph.ainvoke({"messages": [inputs]}, config = configuration)
    
    message = {"role": "assistant", "content": result["messages"][-1].content}
    return message 

Next, for each conversation, we will create a `stopping_condition`. This is an optional step that will allow the simulation determine when to stop, based on the pre-defined criteria

In [6]:
from pydantic import BaseModel, Field
from langchain_core.messages import SystemMessage

class Condition(BaseModel):
    state: bool = Field(description="True if stopping condition was met, False if hasn't been met")

# Define stopping condition 
async def has_satisfied(trajectory, turn_counter):

    structured_llm = model.with_structured_output(schema=Condition)
    structured_system_prompt = """Determine if the stopping condition was met from the following conversation history. 
    To meet the stopping condition, the conversation must follow one of the following scenarios: 
    1. All inquiries are satisfied, and user confirms that there are no additional issues that the support agent can help the customer with. 
    2. Not all user inquiries are satisfied, but next steps are clear, and user confirms that are no other items that the agent can help with. 

    The conversation between the customer and the customer support assistant that you should analyze is as follows:
    {conversation}
    """

    parsed_info = structured_llm.invoke([SystemMessage(content=structured_system_prompt.format(conversation=trajectory))])

    return parsed_info.state

Next, for each **user persona**, we will create a simulated `user` based on our dataset inputs, and run application logic using `run_multiturn_simulation_async`. 

In [7]:
async def run_simulation(inputs: dict):
    # Create a simulated user with seeded messages and system prompt from our dataset
    user = create_llm_simulated_user(
        system=inputs["persona"],
        model="openai:gpt-4.1-mini",
    )

    # Next, let's use openevals to run a simulation with our multiagent
    simulator_result = await run_multiturn_simulation_async(
        app=run_graph,
        user=user,
        max_turns=5,
        stopping_condition=has_satisfied
    )

    # Return the full conversation trajectory as an output
    return {"trajectory": simulator_result["trajectory"]}

#### 3. Define the Evaluator(s)¶

In addition to creating "static" LLM judge prompts that judges user satisfaction and agent professionalism, we will also create an LLM-judge that takes in the success criteria we have defined in reference outputs, and determines if the conversation is resolved based on our defined success criteria. 

In [8]:
# Create evaluators 

prompt = """\n\n Response criteria: {reference_outputs} \n\n 
Assistant's response: \n\n {outputs} \n\n 
Evaluate whether the assistant's response meets the criteria and provide justification for your evaluation."""

resolution_evaluator_async = create_async_llm_as_judge(
    model="openai:gpt-4o-mini",
    prompt="""\n\n Response criteria: {reference_outputs} \n\n Assistant's response: \n\n {outputs} \n\n Evaluate whether the assistant's response meets the criteria and provide justification for your evaluation.""",
    feedback_key="resolution",
)

satisfaction_evaluator_async = create_async_llm_as_judge(
    model="openai:gpt-4o-mini",
    prompt="Based on the below conversation, is the user satisfied?\n{outputs}",
    feedback_key="satisfaction",
)

professionalism_evaluator_async = create_async_llm_as_judge(
    model="openai:gpt-4o-mini",
    prompt="Based on the below conversation, has our agent remained a professional tone throughout the conversation?\n{outputs}",
    feedback_key="professionalism",
)

def num_turns(inputs: dict, outputs: dict, reference_outputs: dict):
    return {"key": "num_turns", "score": (len(outputs["trajectory"])/2)}

#### 4. Run the Evaluation 

In [9]:
experiment_results = await client.aevaluate(
    run_simulation,
    data=dataset_name,
    evaluators=[resolution_evaluator_async,num_turns,satisfaction_evaluator_async,professionalism_evaluator_async],
    experiment_prefix="agent-o3mini-multiturn",
    num_repetitions=1,
    max_concurrency=5,
)

/Users/robertxu/Desktop/Projects/education/eval-concepts/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'agent-o3mini-multiturn-22878134' at:
https://smith.langchain.com/o/58636190-0252-4526-9dc7-0b09b37b499c/datasets/7baa4065-5252-4482-9971-08f3ca0a7e4e/compare?selectedSessions=5f983b2b-0a6c-438a-8605-1a2347164797




0it [00:00, ?it/s]/Users/robertxu/Desktop/Projects/education/eval-concepts/.venv/lib/python3.13/site-packages/pydantic/v1/main.py:1054: UserWarning: LangSmith now uses UUID v7 for run and trace identifiers. This warning appears when passing custom IDs. Please use: from langsmith import uuid7
            id = uuid7()
Future versions will require UUID v7.
  input_data = validator(cls_, input_data)


made it here
invoice subagent input: Provide details of the employee who assisted me with my most recent purchase for customer id 32.
made it here
invoice subagent input: Retrieve the invoice details (invoice ID and amount) for the recent purchase made by customer with ID 30


1it [00:26, 26.97s/it]

made it here
invoice subagent input: Retrieve the transaction date and payment method for invoice ID 333 for customer with ID 30


4it [01:30, 22.53s/it]
